<a href="https://colab.research.google.com/github/squeze/my_udacity_deep_learning_solutions/blob/master/intro-neural-networks/gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing the gradient descent algorithm


This notebook is based on the udacity deep learning nanodegree exercise for gradient descent, which can be found here:

https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-neural-networks/gradient-descent/GradientDescent.ipynb

The original version is implemented with python and numpy, I try to implement it with swift-only as an exercise to learn swift.

## Loading dataset from github
The original dataset is located here:

https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-neural-networks/gradient-descent/data.csv

In [3]:
import Foundation

let url = "https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-neural-networks/gradient-descent/data.csv"

// author of this query function: https://gist.github.com/groz/85b95f663f79ba17946269ea65c2c0f4
func query(address: String) -> String {
    let url = URL(string: address)
    let semaphore = DispatchSemaphore(value: 0)
    
    var result: String = ""
    
    let task = URLSession.shared.dataTask(with: url!) {(data, response, error) in
        result = String(data: data!, encoding: String.Encoding.utf8)!
        semaphore.signal()
    }
    
    task.resume()
    semaphore.wait()
    return result
}

let rawData = query(address: url)
let rows = rawData.components(separatedBy: "\n")
let featuresAndTargets = rows.map({ $0.components(separatedBy: ",") })
print(featuresAndTargets)

[["0.78051", "-0.063669", "1"], ["0.28774", "0.29139", "1"], ["0.40714", "0.17878", "1"], ["0.2923", "0.4217", "1"], ["0.50922", "0.35256", "1"], ["0.27785", "0.10802", "1"], ["0.27527", "0.33223", "1"], ["0.43999", "0.31245", "1"], ["0.33557", "0.42984", "1"], ["0.23448", "0.24986", "1"], ["0.0084492", "0.13658", "1"], ["0.12419", "0.33595", "1"], ["0.25644", "0.42624", "1"], ["0.4591", "0.40426", "1"], ["0.44547", "0.45117", "1"], ["0.42218", "0.20118", "1"], ["0.49563", "0.21445", "1"], ["0.30848", "0.24306", "1"], ["0.39707", "0.44438", "1"], ["0.32945", "0.39217", "1"], ["0.40739", "0.40271", "1"], ["0.3106", "0.50702", "1"], ["0.49638", "0.45384", "1"], ["0.10073", "0.32053", "1"], ["0.69907", "0.37307", "1"], ["0.29767", "0.69648", "1"], ["0.15099", "0.57341", "1"], ["0.16427", "0.27759", "1"], ["0.33259", "0.055964", "1"], ["0.53741", "0.28637", "1"], ["0.19503", "0.36879", "1"], ["0.40278", "0.035148", "1"], ["0.21296", "0.55169", "1"], ["0.48447", "0.56991", "1"], ["0.25476",

## Sigmoid activation function
$$\sigma(x) = \frac{1}{1+e^{-x}}$$

Swift is strongly typed, types for input/output must be given. Sigmoid must handle a tensor of floating point values. Choose double.

Swift uses local and external parameter names. To avoid giving the name of the parameter by calling the function, use _ for the external name.

In [0]:
import TensorFlow

func mySigmoid(_ x: Tensor<Double>) -> Tensor<Double> {
  return 1 / (1 + exp(-x))
}

### Test
Test with original tensorflow sigmoid function

In [39]:
let original = sigmoid(Tensor([1.0]))
let myResult = mySigmoid(Tensor([1.0]))
print(original[0], myResult[0])
assert(original[0] == myResult[0])

0.7310585786300049 0.7310585786300049
